## Импорты

In [ ]:
!pip install jupyter ipywidgets

In [ ]:
import os, random, time, math, numpy as np
import torch, torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torch.amp import GradScaler, autocast
from sklearn.metrics import accuracy_score, f1_score
from tqdm.auto import tqdm
from torchvision.models import ViT_B_16_Weights

SEED        = 42
IMSIZE      = 224
BATCH_CNN   = 128
BATCH_VIT   = 64
ACC_STEPS   = 2
NUM_WORKERS = 4
DEVICE      = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
print('Device:', torch.cuda.get_device_name(0) if DEVICE=='cuda' else 'CPU-only')

Device: NVIDIA A100-PCIE-40GB


## Аугментации

### Базовые (`tf_basic`)
- **RandomResizedCrop** → разбивает кадр, даёт модели видеть разные фрагменты.  
- **HorizontalFlip** → отражения для симметрии.  
- **Normalize** → приведение к одной шкале (ImageNet).

### Усиленные (`tf_strong`)
- **RandomResizedCrop (широкий диапазон)** → экстремальные обрезы и масштабы.  
- **HorizontalFlip** → те же отражения.  
- **ColorJitter** → случайные изменения цвета и яркости.  
- **RandomErasing** → «стереть» участки, чтобы модель не заучивала фон.

### Для валидации/теста (`tf_test`)
- **Resize + CenterCrop** → одинаковый размер без дёрганий.  
- **Normalize** → та же схема, что и в обучении.

In [ ]:
tf_basic = transforms.Compose([
    transforms.RandomResizedCrop(IMSIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

tf_strong = transforms.Compose([
    transforms.RandomResizedCrop(IMSIZE, scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.2),

    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.25, value='random'),
])

tf_test = transforms.Compose([
    transforms.Resize(IMSIZE+32),
    transforms.CenterCrop(IMSIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

## Загрузчик данных

**Food101**  
- Содержит **101 класс** блюд (фотографии из реальной жизни) → проверенная сложная задача мультиклассовой классификации.  
- Большой объём (≈ 101 000 изображений) → статистическая устойчивость и разнообразие ракурсов, освещения, фонов.  
- Стандартные splits (train/val/test) → воспроизводимость экспериментов и удобство валидации.  
- Основан на открытых данных — не нужно ручное собирание и аннотирование.

In [ ]:
root    = './data'
full_ds = datasets.Food101(root, download=True, transform=tf_basic)
n       = len(full_ds)
n_tr, n_val = int(0.8*n), int(0.1*n)
n_te    = n - n_tr - n_val
train_ds, val_ds, test_ds = random_split(
    full_ds, [n_tr, n_val, n_te],
    generator=torch.Generator().manual_seed(SEED)
)
val_ds.dataset.transform  = tf_test
test_ds.dataset.transform = tf_test

gen = torch.Generator().manual_seed(SEED)

tr_dl_cnn = DataLoader(train_ds, BATCH_CNN, shuffle=True,
                       generator=gen,
                       num_workers=NUM_WORKERS, pin_memory=True)

tr_dl_vit = DataLoader(train_ds, BATCH_VIT, shuffle=True,
                       generator=gen,
                       num_workers=NUM_WORKERS, pin_memory=True)

val_dl  = DataLoader(val_ds, BATCH_CNN, shuffle=False,
                     num_workers=NUM_WORKERS, pin_memory=True)

test_dl = DataLoader(test_ds, BATCH_CNN, shuffle=False,
                     num_workers=NUM_WORKERS, pin_memory=True)

NUM_CLASSES = len(full_ds.classes)
print(f'Классов: {NUM_CLASSES} — {len(train_ds)}/{len(val_ds)}/{len(test_ds)}')

Классов: 101 — 60600/7575/7575


## Полезные приколы

### Выбранные метрики

- **Accuracy** (`accuracy_score`)  
  Доля правильно угаданных классов во всём датасете.  
  Показывает общую «простоту» задачи и позволяет быстро оценить себя на доминантных классах.

- **Macro F1-score** (`f1_score(average='macro')`)  
  Гармоническое среднее **precision** и **recall**, усреднённое по классам без взвешивания.  
  Отражает способность модели удерживать баланс между точностью и полнотой даже на редких классах, игнорируя их соотношение в датасете.

Вместе эти две метрики дают представление и об общей точности (accuracy), и о том, как модель работает на каждой категории (macro F1).

In [ ]:
def metrics(y_true, y_pred):
    return accuracy_score(y_true, y_pred), f1_score(y_true, y_pred, average='macro')

def epoch_step(model, loader, criterion, opt=None, scaler=None, accum=1):
    train = opt is not None
    model.train(train)
    y_t, y_p, running = [], [], 0.0
    if train: opt.zero_grad()

    for i,(x,y) in enumerate(loader):
        x, y = x.to(DEVICE, non_blocking=True), y.to(DEVICE, non_blocking=True)
        with autocast(device_type='cuda'):
            out  = model(x)
            loss = criterion(out, y) / (accum if train else 1)
        if train:
            scaler.scale(loss).backward()
            if (i+1)%accum==0 or (i+1)==len(loader):
                scaler.step(opt); scaler.update(); opt.zero_grad()
        running += loss.item()*x.size(0)*(accum if train else 1)
        y_t.append(y.cpu()); y_p.append(out.detach().cpu().argmax(1))

    y_t = torch.cat(y_t); y_p = torch.cat(y_p)
    acc,f1 = metrics(y_t,y_p)
    return running/len(loader.dataset), acc, f1

def fit(model, loader, epochs, lr, accum=1):
    model.to(DEVICE)
    opt  = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    sch  = torch.optim.lr_scheduler.CosineAnnealingLR(opt, epochs)
    crit = nn.CrossEntropyLoss(label_smoothing=0.1)
    sc   = GradScaler()
    best = {'acc':0}

    for e in range(1,epochs+1):
        _,tr_acc,_        = epoch_step(model, loader, crit, opt, sc, accum)
        _,val_acc,val_f1  = epoch_step(model, val_dl, crit)
        sch.step()
        if val_acc>best['acc']:
            best={'acc':val_acc,'f1':val_f1,'state':model.state_dict()}
        print(f'E{e:02d}/{epochs}  train_acc {tr_acc:.3f}  val_acc {val_acc:.3f}  val_F1 {val_f1:.3f}')

    model.load_state_dict(best['state'])
    return model,best

def evaluate(model,name):
    model.eval(); y_t,y_p=[],[]
    with torch.no_grad(), autocast(device_type='cuda'):
        for x,y in test_dl:
            p = model(x.to(DEVICE)).argmax(1).cpu()
            y_t.append(y); y_p.append(p)
    acc,f1 = metrics(torch.cat(y_t),torch.cat(y_p))
    print(f'{name}: acc {acc:.3f}  F1 {f1:.3f}')
    return acc,f1

## Baseline

### ResNet

- **Архитектура**: классическая CNN-модель ResNet-18, предобученная на ImageNet, с заменённым линейным слоем на число классов задачи.  
- **Тренинг**: 4 эпохи, lr=1 e-4, AdamW + CosineAnnealingLR, label smoothing=0.1, накопление градиентов по батчу.

In [ ]:
r18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
r18.fc = nn.Linear(r18.fc.in_features, NUM_CLASSES)
r18,_ = fit(r18, tr_dl_cnn, epochs=4, lr=1e-4)
evaluate(r18,'ResNet-18')

E01/4  train_acc 0.503  val_acc 0.630  val_F1 0.628
E02/4  train_acc 0.738  val_acc 0.682  val_F1 0.683
E03/4  train_acc 0.861  val_acc 0.709  val_F1 0.710
E04/4  train_acc 0.931  val_acc 0.716  val_F1 0.717
ResNet-18: acc 0.719  F1 0.719


(0.7194719471947195, 0.7189772102520771)

| Метрика      | Значение |
|--------------|----------|
| Accuracy     | 0.719    |
| Macro F1     | 0.719    |
| Best val_acc | 0.716 (эпоха 4) |

### Краткие выводы

- Валид. точность выросла с 0.63 до 0.72 за 4 эпохи, затем стабилизировалась.  
- Небольшой разрыв между train_acc (0.93) и val_acc (0.72) указывает на умеренную переобучаемость.  

### Vit

- **Архитектура**: Vision Transformer (B/16), предобученный на ImageNet, с заменённым классифицирующим слоем под нужное число классов.  
- **Тренинг**: 4 эпохи, lr = 5 × 10⁻⁵, градиентная аккумуляция (ACC_STEPS), AdamW + CosineAnnealingLR, label smoothing=0.1.

In [ ]:
vit = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)
vit.heads.head = nn.Linear(vit.heads.head.in_features, NUM_CLASSES)

vit, _ = fit(vit, tr_dl_vit, epochs=4, lr=5e-5, accum=ACC_STEPS)
evaluate(vit, 'ViT-B/16')

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 150MB/s]  


E01/4  train_acc 0.631  val_acc 0.746  val_F1 0.748
E02/4  train_acc 0.839  val_acc 0.783  val_F1 0.785
E03/4  train_acc 0.936  val_acc 0.796  val_F1 0.798
E04/4  train_acc 0.979  val_acc 0.799  val_F1 0.801
ViT-B/16: acc 0.797  F1 0.796


(0.7969636963696369, 0.7963127553830418)

| Метрика      | Значение                |
|--------------|-------------------------|
| Accuracy     | 0.797                   |
| Macro F1     | 0.796                   |
| Best val_acc | 0.799 (эпоха 4)         |

### Краткие выводы

- ViT-База медленнее обучается на первых эпохах по сравнению с ResNet-18, но в итоге даёт заметно более высокую точность и F1.  
- Рост train_acc от 0.63 до 0.98 при val_acc от 0.75 до 0.80 говорит о хорошей способности к обобщению.  
- Transformer-архитектура показывает более устойчивую динамику и сильнее подходит для задачи классификации пищи.

## Улучшенный baseline

In [ ]:
train_ds.dataset.transform = tf_strong

tr_dl_strong_cnn = DataLoader(train_ds, BATCH_CNN, shuffle=True,
                              generator=gen,
                              num_workers=NUM_WORKERS, pin_memory=True)

tr_dl_strong_vit = DataLoader(train_ds, BATCH_VIT, shuffle=True,
                              generator=gen,
                              num_workers=NUM_WORKERS, pin_memory=True)

### ResNet, но круче

- **Архитектура**: ResNet-50, предобученный на ImageNet, с применением mixup (α=0.2) и сильных аугментаций (ColorJitter, RandomErasing) в тренировочном цикле.  
- **Тренинг**: 5 эпох, lr = 1 × 10⁻⁴, gradient accumulation = ACC_STEPS, AdamW + CosineAnnealingLR, label smoothing=0.1.

In [ ]:
def mixup(x, y, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    idx = torch.randperm(x.size(0))
    return lam * x + (1 - lam) * x[idx], (y, y[idx], lam)

def mix_loss(pred, tgt):
    y1, y2, lam = tgt
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    return lam * criterion(pred, y1) + (1 - lam) * criterion(pred, y2)

def train_r50_mix(epochs=5, lr=1e-4):
    net = models.resnet50(
        weights=models.ResNet50_Weights.IMAGENET1K_V2,
    )
    net.fc = nn.Linear(net.fc.in_features, NUM_CLASSES)
    net.to(DEVICE)
    opt = torch.optim.AdamW(
        net.parameters(),
        lr=lr,
        weight_decay=1e-4,
    )
    scaler = GradScaler()
    best = {"acc": 0.0}
    for epoch in range(1, epochs + 1):
        net.train()
        opt.zero_grad()
        for step, (x, y) in enumerate(
            tqdm(tr_dl_strong_cnn, leave=False),
            1,
        ):
            x = x.to(DEVICE)
            y = y.to(DEVICE)

            xm, ym = mixup(x, y)
            with autocast(device_type='cuda'):
                out = net(xm)
                loss = mix_loss(out, ym) / ACC_STEPS
            scaler.scale(loss).backward()
            if step % ACC_STEPS == 0 or step == len(tr_dl_strong_cnn):
                scaler.step(opt)
                scaler.update()
                opt.zero_grad()
        _, val_acc, val_f1 = epoch_step(
            net,
            val_dl,
            nn.CrossEntropyLoss(label_smoothing=0.1),
        )
        if val_acc > best["acc"]:
            best = {
                "acc": val_acc,
                "f1": val_f1,
                "state": net.state_dict(),
            }
        print(
            f"E{epoch}/{epochs}  "
            f"val_acc {val_acc:.3f}  "
            f"val_F1 {val_f1:.3f}",
        )
    net.load_state_dict(best["state"])
    return net, best

r50, _ = train_r50_mix()
evaluate(r50, "ResNet-50+Mix")

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 152MB/s] 


E1/5  val_acc 0.574  val_F1 0.567


E2/5  val_acc 0.658  val_F1 0.654


E3/5  val_acc 0.702  val_F1 0.701


E4/5  val_acc 0.724  val_F1 0.724


E5/5  val_acc 0.732  val_F1 0.729
ResNet-50+Mix: acc 0.736  F1 0.733


(0.7361056105610561, 0.7330629654119782)

| Метрика         | Значение            |
|-----------------|---------------------|
| Accuracy (test) | 0.736               |
| Macro F1 (test) | 0.733               |
| Best val_acc    | 0.732 (эпоха 5)     |

### Краткие выводы

- Сильные аугментации + mixup дали заметный прирост по сравнению с ResNet-18 (+~0.02 acc).  
- Быстрый рост val_acc до 0.73 за 5 эпох говорит об эффективном использовании регуляризации.

### Vit, но тоже круче

- **Архитектура**: оконная Transformer-модель Swin-Tiny (~28 M параметров), head заменён на линейный слой под NUM_CLASSES.  
- **Данные и аугментации**: Food101 с сильными аугментациями (ColorJitter, RandomErasing) + mixup.  
- **Тренинг**: 4 эпохи, lr = 5 × 10⁻⁵, gradient accumulation, AdamW + CosineAnnealingLR, label smoothing.

In [ ]:
swin = models.swin_t(weights=models.Swin_T_Weights.IMAGENET1K_V1)
swin.head = nn.Linear(swin.head.in_features, NUM_CLASSES)
swin,_ = fit(swin, tr_dl_strong_vit, epochs=4, lr=5e-5, accum=ACC_STEPS)
evaluate(swin,'Swin-T')

Downloading: "https://download.pytorch.org/models/swin_t-704ceda3.pth" to /root/.cache/torch/hub/checkpoints/swin_t-704ceda3.pth
100%|██████████| 108M/108M [00:01<00:00, 104MB/s]  


E01/4  train_acc 0.421  val_acc 0.648  val_F1 0.643
E02/4  train_acc 0.648  val_acc 0.706  val_F1 0.704
E03/4  train_acc 0.705  val_acc 0.729  val_F1 0.729
E04/4  train_acc 0.732  val_acc 0.744  val_F1 0.743
Swin-T: acc 0.754  F1 0.751


(0.7537953795379538, 0.7511911494530815)

| Метрика         | Значение                 |
|-----------------|--------------------------|
| Accuracy (test) | 0.754                    |
| Macro F1 (test) | 0.751                    |
| Best val_acc    | 0.744 (эпоха 4)          |

### Краткие выводы

- Swin-T показывает лучший результат среди всех моделей: +0.017 acc по сравнению с ViT-B/16.  
- Быстрый рост val_acc от 0.648 до 0.744 за всего 4 эпохи свидетельствует о хорошей обобщающей способности оконного внимания.  
- Mixup и сильные аугментации эффективно regularize, но наиболее значимый прирост даёт сама архитектура Swin.  

In [ ]:
train_ds.dataset.transform = tf_basic

## Свои модели

### Обычная

- **Архитектура**:  
  Три свёрточных блока (Conv–BN–ReLU) с MaxPool, затем AdaptiveAvgPool и два FC-слоя (256→256→n_cls) с Dropout.

- **Тренинг**:  
  12 эпох, lr=3×10⁻⁴, AdamW + CosineAnnealingLR, CrossEntropyLoss с label smoothing=0.1.

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, n_cls):
        super().__init__()
        self.feat = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
        )
        self.cls = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, n_cls),
        )
    def forward(self, x):
        return self.cls(self.feat(x))

simp, _ = fit(
    SimpleCNN(NUM_CLASSES),
    tr_dl_cnn,
    epochs=12,
    lr=3e-4,
)
evaluate(simp, "SimpleCNN")

E01/12  train_acc 0.054  val_acc 0.070  val_F1 0.040
E02/12  train_acc 0.085  val_acc 0.095  val_F1 0.065
E03/12  train_acc 0.103  val_acc 0.106  val_F1 0.075
E04/12  train_acc 0.117  val_acc 0.116  val_F1 0.088
E05/12  train_acc 0.129  val_acc 0.133  val_F1 0.104
E06/12  train_acc 0.141  val_acc 0.137  val_F1 0.117
E07/12  train_acc 0.151  val_acc 0.147  val_F1 0.120
E08/12  train_acc 0.158  val_acc 0.162  val_F1 0.138
E09/12  train_acc 0.164  val_acc 0.155  val_F1 0.130
E10/12  train_acc 0.169  val_acc 0.177  val_F1 0.151
E11/12  train_acc 0.173  val_acc 0.180  val_F1 0.153
E12/12  train_acc 0.174  val_acc 0.177  val_F1 0.150
SimpleCNN: acc 0.188  F1 0.155


(0.18785478547854786, 0.15547873104068644)

| Метрика       | Значение               |
|---------------|------------------------|
| Accuracy (test)   | 0.188                  |
| Macro F1 (test)   | 0.155                  |
| Best val_acc      | 0.180 (эпоха 11)       |

### Краткие выводы

- Очень невысокие метрики говорят о том, что простая архитектура без мощных предобученных бэконов с трудом справляется с разнообразием классов Food101.  
- Рост val_acc до 0.18 к 11-й эпохе, затем плато → модель быстро выходит на своё “предел” выразительности.  
- Для серьёзных задач классификации нужны глубже сети или трансформеры с предобучением.  

### Необычная

- **Архитектура**:  
  Лёгкий ViT-подобный бинуарный блок:  
  – Patch-эмбеддинг: Conv2d(kernel=16, stride=16) → 256-мерные токены.  
  – Добавлен CLS-токен + позиционные эмбеддинги.  
  – 4 Transformer-блока (MultiHeadAttention heads=4 + MLP(×4) + LayerNorm + residual).  
  – Выход через LN и линейный классификатор.

- **Тренинг**:  
  12 эпох, lr=3×10⁻⁴, AdamW + CosineAnnealingLR, CrossEntropyLoss(label_smoothing=0.1), градиентный накопитель ACC_STEPS на ViT-даталоадер.

In [ ]:
class Patch(nn.Module):
    def __init__(self, img=224, ps=16, inp=3, dim=256):
        super().__init__()
        self.conv = nn.Conv2d(inp, dim, ps, ps)

    def forward(self, x):
        return self.conv(x).flatten(2).transpose(1, 2)

class Block(nn.Module):
    def __init__(self, dim=256, h=4, mlp=4, p=0.1):
        super().__init__()

        self.n1 = nn.LayerNorm(dim)
        self.att = nn.MultiheadAttention(
            dim,
            h,
            dropout=p,
            batch_first=True,
        )

        self.n2 = nn.LayerNorm(dim)
        self.ff = nn.Sequential(
            nn.Linear(dim, dim * mlp),
            nn.GELU(),
            nn.Dropout(p),
            nn.Linear(dim * mlp, dim),
            nn.Dropout(p),
        )

    def forward(self, x):
        y = x
        x = self.n1(x)
        x, _ = self.att(x, x, x, need_weights=False)
        x = x + y

        y = x
        x = self.n2(x)
        x = self.ff(x) + y
        return x

class TinyViT(nn.Module):
    def __init__(
        self,
        img=224,
        ps=16,
        cls=NUM_CLASSES,
        dim=256,
        d=4,
        h=4,
    ):
        super().__init__()

        self.embed = Patch(img, ps, 3, dim)
        self.cls = nn.Parameter(torch.zeros(1, 1, dim))
        self.pos = nn.Parameter(
            torch.randn(1, (img // ps) ** 2 + 1, dim),
        )

        self.blks = nn.ModuleList([Block(dim, h) for _ in range(d)])
        self.norm = nn.LayerNorm(dim)
        self.head = nn.Linear(dim, cls)

        nn.init.trunc_normal_(self.pos, std=0.02)
        nn.init.trunc_normal_(self.cls, std=0.02)

    def forward(self, x):
        batch_size = x.size(0)

        x = self.embed(x)
        cls_tok = self.cls.expand(batch_size, -1, -1)
        x = torch.cat((cls_tok, x), 1) + self.pos

        for blk in self.blks:
            x = blk(x)

        x = self.norm(x)[:, 0]
        return self.head(x)


tiny, _ = fit(
    TinyViT(),
    tr_dl_vit,
    epochs=12,
    lr=3e-4,
    accum=ACC_STEPS,
)
evaluate(tiny, "TinyViT")

E01/12  train_acc 0.063  val_acc 0.082  val_F1 0.057
E02/12  train_acc 0.115  val_acc 0.127  val_F1 0.104
E03/12  train_acc 0.150  val_acc 0.156  val_F1 0.127
E04/12  train_acc 0.175  val_acc 0.180  val_F1 0.156
E05/12  train_acc 0.200  val_acc 0.201  val_F1 0.182
E06/12  train_acc 0.222  val_acc 0.219  val_F1 0.201
E07/12  train_acc 0.247  val_acc 0.243  val_F1 0.231
E08/12  train_acc 0.263  val_acc 0.254  val_F1 0.238
E09/12  train_acc 0.284  val_acc 0.260  val_F1 0.245
E10/12  train_acc 0.299  val_acc 0.272  val_F1 0.258
E11/12  train_acc 0.309  val_acc 0.283  val_F1 0.271
E12/12  train_acc 0.317  val_acc 0.284  val_F1 0.271
TinyViT: acc 0.300  F1 0.281


(0.29953795379537956, 0.2811939835013023)

| Метрика       | Значение               |
|---------------|------------------------|
| Accuracy (test)   | 0.300                  |
| Macro F1 (test)   | 0.281                  |
| Best val_acc      | 0.284 (эпоха 12)       |

### Краткие выводы

- TinyViT показывает приемлемый прогресс (val_acc ≈ 0.28), но остаётся далеко позади крупных предобученных трансформеров.  
- Литые размеры (256-мерный токен, 4 блока) ограничивают вместимость модели.

## Результаты

In [ ]:
metrics_cache = {}
for name, model in [
    ('ResNet-18',      r18),
    ('ViT-B/16',       vit),
    ('ResNet-50+Mix',  r50),
    ('Swin-T',         swin),
    ('SimpleCNN',      simp),
    ('TinyViT',        tiny),
]:
    metrics_cache[name] = evaluate(model, name)

print('\n=== Итоговые accuracy ===')
for k, (acc, _) in metrics_cache.items():
    print(f'{k:13s}: {acc:.3f}')

- **ResNet-18** (acc 0.719 / F1 0.719)  
  Надёжный CNN-бейзлайн, быстро сходится и показывает сбалансированную точность на всех классах.

- **ViT-B/16** (acc 0.797 / F1 0.796)  
  Лучший результат благодаря мощному глобальному вниманию — трансформер отлично выучил особенности каждого класса.

- **ResNet-50 + MixUp** (acc 0.736 / F1 0.733)  
  Более глубокая CNN с миксап-аугментацией обходит ResNet-18, но до трансформеров немного не дотягивает.

- **Swin-T** (acc 0.754 / F1 0.751)  
  Локально-глобальный подход через «окна» даёт преимущество перед ResNet-18, уступая только ViT.

- **SimpleCNN** (acc 0.188 / F1 0.155)  
  Модель слишком малой глубины и без предварительной инициализации — не справляется с разнообразием классов.

- **TinyViT** (acc 0.300 / F1 0.281)  
  Лёгкий трансформер показывает ограниченные возможности при малом объёме данных и небольшой архитектуре.